In [60]:
import boto3
import sagemaker
import datetime as dt
import pandas as pd

In [61]:
#Replace this value with the S3 Bucket Created
default_bucket = "cisco-churn-project-data"

In [62]:
region = boto3.Session().region_name
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
sklearn_processor_version="0.23-1"
model_package_group_name="ChurnModelPackageGroup"
pipeline_name= "ChurnModelSMPipeline"
clarify_image = sagemaker.image_uris.retrieve(framework='sklearn', version=sklearn_processor_version, region=region)
# Sanity check
print('region:', pipeline_name)

region: ChurnModelSMPipeline


In [63]:
# Step 2: Generate Baseline Dataset

In [64]:
def preprocess_data(file_path):
    df = pd.read_csv(file_path)
    ## Convert to datetime columns
    df["firstorder"]=pd.to_datetime(df["firstorder"],errors='coerce')
    df["lastorder"] = pd.to_datetime(df["lastorder"],errors='coerce')
    ## Drop Rows with null values
    df = df.dropna()
    ## Create Column which gives the days between the last order and the first order
    df["first_last_days_diff"] = (df['lastorder']-df['firstorder']).dt.days
    ## Create Column which gives the days between when the customer record was created and the first order
    df['created'] = pd.to_datetime(df['created'])
    df['created_first_days_diff']=(df['created']-df['firstorder']).dt.days
    ## Drop Columns
    df.drop(['custid','created','firstorder','lastorder'],axis=1,inplace=True)
    ## Apply one hot encoding on favday and city columns
    df = pd.get_dummies(df,prefix=['favday','city'],columns=['favday','city'])
    return df

In [65]:
baseline_data = preprocess_data("data/storedata_total.csv")
baseline_data.pop("retained")
baseline_sample = baseline_data.sample(frac=0.0002)

In [66]:
pd.DataFrame(baseline_sample).to_csv("data/baseline.csv",header=False,index=False)

In [ ]:
# Step 3: Generate Batch Dataset

In [67]:
batch_data = preprocess_data("data/storedata_total.csv")
batch_data.pop("retained")
batch_sample = batch_data.sample(frac=0.2)

In [68]:
pd.DataFrame(batch_sample).to_csv("data/batch.csv",header=False,index=False)

In [69]:
# Step 4: Copy Data and Scripts to S3 Bucket

In [76]:
s3_client = boto3.resource('s3')
s3_client.Bucket(default_bucket).upload_file("data/storedata_total.csv","data/storedata_total.csv")
s3_client.Bucket(default_bucket).upload_file("data/batch.csv","data/batch/batch.csv")
s3_client.Bucket(default_bucket).upload_file("data/baseline.csv","input/baseline/baseline.csv")

In [79]:
s3_client.Bucket(default_bucket).upload_file("pipelines/customerchurn/preprocess.py","input/code/preprocess.py")
s3_client.Bucket(default_bucket).upload_file("pipelines/customerchurn/evaluate.py","input/code/evaluate.py")
s3_client.Bucket(default_bucket).upload_file("pipelines/customerchurn/generate_config.py","input/code/generate_config.py")

In [81]:
from pipelines.customerchurn.pipeline import get_pipeline

pipeline = get_pipeline(
    region=region,
    role=role,
    default_bucket=default_bucket,
    model_package_group_name=model_package_group_name,
    pipeline_name=pipeline_name,
    custom_image_uri=clarify_image,
    sklearn_processor_version=sklearn_processor_version
)

INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: 1.0.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [82]:
pipeline.definition()

'{"Version": "2020-12-01", "Metadata": {}, "Parameters": [{"Name": "ProcessingInstanceType", "Type": "String", "DefaultValue": "ml.m5.xlarge"}, {"Name": "ProcessingInstanceCount", "Type": "Integer", "DefaultValue": 1}, {"Name": "TrainingInstanceType", "Type": "String", "DefaultValue": "ml.m5.xlarge"}, {"Name": "InputData", "Type": "String", "DefaultValue": "s3://cisco-churn-project-data/data/storedata_total.csv"}, {"Name": "BatchData", "Type": "String", "DefaultValue": "s3://cisco-churn-project-data/data/batch/batch.csv"}], "PipelineExperimentConfig": {"ExperimentName": {"Get": "Execution.PipelineName"}, "TrialName": {"Get": "Execution.PipelineExecutionId"}}, "Steps": [{"Name": "ChurnModelProcess", "Type": "Processing", "Arguments": {"ProcessingResources": {"ClusterConfig": {"InstanceType": {"Get": "Parameters.ProcessingInstanceType"}, "InstanceCount": {"Get": "Parameters.ProcessingInstanceCount"}, "VolumeSizeInGB": 30}}, "AppSpecification": {"ImageUri": "257758044811.dkr.ecr.us-east-2

In [ ]:
# Step 5: Submit the pipeline to SageMaker and start execution

In [83]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-2:961341547991:pipeline/ChurnModelSMPipeline',
 'ResponseMetadata': {'RequestId': '0f6d8dca-cfb5-4a80-8fbe-70760a8c1928',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0f6d8dca-cfb5-4a80-8fbe-70760a8c1928',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '88',
   'date': 'Wed, 22 Jan 2025 03:06:06 GMT'},
  'RetryAttempts': 0}}

In [84]:
execution = pipeline.start()

In [85]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-2:961341547991:pipeline/ChurnModelSMPipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-2:961341547991:pipeline/ChurnModelSMPipeline/execution/u62c2uf1ldig',
 'PipelineExecutionDisplayName': 'execution-1737515180462',
 'PipelineExecutionStatus': 'Failed',
 'PipelineExperimentConfig': {'ExperimentName': 'churnmodelsmpipeline',
  'TrialName': 'u62c2uf1ldig'},
 'FailureReason': 'Step failure: One or multiple steps failed.',
 'CreationTime': datetime.datetime(2025, 1, 22, 3, 6, 20, 351000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 1, 22, 3, 6, 23, 175000, tzinfo=tzlocal()),
 'CreatedBy': {'IamIdentity': {'Arn': 'arn:aws:sts::961341547991:assumed-role/AmazonSageMaker-ExecutionRole-20250109T202735/SageMaker',
   'PrincipalId': 'AROA57VDL4XLQPNELYXLL:SageMaker'}},
 'LastModifiedBy': {'IamIdentity': {'Arn': 'arn:aws:sts::961341547991:assumed-role/AmazonSageMaker-ExecutionRole-20250109T202735/SageMaker',
   'PrincipalId

In [86]:
execution.list_steps()

[{'StepName': 'ChurnModelProcess',
  'StartTime': datetime.datetime(2025, 1, 22, 3, 6, 21, 341000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2025, 1, 22, 3, 6, 22, 870000, tzinfo=tzlocal()),
  'StepStatus': 'Failed',
  'FailureReason': "ClientError: Failed to invoke sagemaker:CreateProcessingJob. Error Details: The account-level service limit 'ml.m5.xlarge for processing job usage' is 0 Instances, with current utilization of 0 Instances and a request delta of 1 Instances. Please use AWS Service Quotas to request an increase for this quota. If AWS Service Quotas is not available, contact AWS support to request an increase for this quota.\nRetry not appropriate on execution of step with PipelineExecutionArn arn:aws:sagemaker:us-east-2:961341547991:pipeline/churnmodelsmpipeline/execution/u62c2uf1ldig and StepId ChurnModelProcess. No retry policy configured for the exception type SAGEMAKER_RESOURCE_LIMIT.",
  'Metadata': {},
  'AttemptCount': 1}]